In [15]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import lightgbm as lgbm
import catboost
from catboost import CatBoostClassifier
from ast import literal_eval

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tqdm import tqdm

import sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score
from sklearn.model_selection import KFold, TimeSeriesSplit, StratifiedKFold, GroupKFold, train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras import optimizers
from keras.models import load_model

import itertools

import datetime
import time
from tqdm import tqdm

import gc

In [153]:
def get_map(data):
    train = data

    map = pd.DataFrame()
    map['player0_start'] = ""
    map['player1_start'] = ""



    for j in tqdm(range(len(train))):
        a = []
        b = []
        for i in range(len(train.loc[j,'player0_event_Camera'])):
            if(i>2):
                if(train.loc[j,'player0_event_Camera'][i] == "'"):
                    break
                elif(i>4):
                  a.append(train.loc[j,'player0_event_Camera'][i])
        
        for i in range(len(train.loc[j,'player1_event_Camera'])):
            if(i>2):
                if(train.loc[j,'player1_event_Camera'][i] == "'"):
                    break
                elif(i>4):
                  b.append(train.loc[j,'player1_event_Camera'][i])
        list = ''.join(a) 
        list2 = ''.join(b)
        map.loc[j] = [list,list2]


    map['player0_start'] = map.player0_start.str.split('(').str[1]
    map['player0_start'] = map.player0_start.str.split(')').str[0]
    split_xy = map.player0_start.str.split(',')
    map['player0_x'] = split_xy.str[0].astype('float')
    map['player0_y'] = split_xy.str[1].astype('float')
    del split_xy

    map['player1_start'] = map.player1_start.str.split('(').str[1]
    map['player1_start'] = map.player1_start.str.split(')').str[0]
    split_xy = map.player1_start.str.split(',')
    map['player1_x'] = split_xy.str[0].astype('float')
    map['player1_y'] = split_xy.str[1].astype('float')
    del split_xy
    

    location_p0 = map.loc[:, ['player0_x', 'player0_y']]
    location_p0 = location_p0.rename({'player0_x':'location_x', 'player0_y':'location_y'}, axis=1)

    location_p1 = map.loc[:, ['player1_x', 'player1_y']]
    location_p1 = location_p1.rename({'player1_x':'location_x', 'player1_y':'location_y'}, axis=1)
    location_p1.index += location_p0.index[-1]+1

    location = pd.concat([location_p0, location_p1])
    location = location.dropna()
    del location_p0, location_p1

    kmeans_clst = KMeans(n_clusters=15).fit(location)
    location['starting'] = kmeans_clst.labels_+1

    for cluster in range(15):
        point = location[location.starting==cluster+1]
        loc = point.loc[:,['location_x', 'location_y']]
        del point
        loc['center_x'] = kmeans_clst.cluster_centers_[cluster][0]
        loc['center_y'] = kmeans_clst.cluster_centers_[cluster][1]
        distance = np.sqrt(np. square(loc.location_x - loc.center_x) + np.square(loc.location_y - loc.center_y))
        location.loc[loc.index, 'distance'] = distance
        del loc
    
    idx = location[location.distance>5].index
    location.loc[idx, 'starting'] = 0
    del idx


    map['player0_start'] = location['starting']
    location.index -= (map.index[-1]+1)
    map['player1_start'] = location['starting']
    del location


    map = map.drop(['player0_x', 'player0_y', 'player1_x', 'player1_y'], axis = 1)
    map = map.fillna(0)

    map_list = []
    for point in range(1,16):
        try:
            couple = map[map.player0_start == point].player1_start.value_counts()
            if couple[couple.index[1]]<100:
                map_list.append([point, couple.index[0], 999])
            else:
                map_list.append([point, couple.index[0], couple.index[1]])
        except Exception as e:
            map_list.append([point, couple.index[0], 999])
            
    map_list = np.sort(map_list, axis = 1)
    map_list = np.unique(map_list, axis = 0)

    for m in map_list:
        idx = map[(map.player0_start == 0)&((map.player1_start == m[0])|(map.player1_start == m[2]))].index
        map.loc[idx, 'player0_start'] = m[1]
        del idx
        idx = map[(map.player0_start == 0)&((map.player1_start == m[1])|(map.player1_start == m[2]))].index
        map.loc[idx, 'player0_start'] = m[0]
        del idx
    
        idx = map[(map.player1_start == 0)&((map.player0_start == m[0])|(map.player0_start == m[2]))].index
        map.loc[idx, 'player1_start'] = m[1]
        del idx
        idx = map[(map.player1_start == 0)&((map.player0_start == m[1])|(map.player0_start == m[2]))].index
        map.loc[idx, 'player1_start'] = m[0]
        del idx
    
    for map_num, m in enumerate(map_list):
        idx = map[(map.player0_start == m[0])|(map.player0_start == m[1])|(map.player0_start == m[2])].index
        map.loc[idx, 'map'] = map_num
    del idx, map_list
    return map

In [154]:
#first_X_test = pd.read_csv('./data/train/first_X_test.csv')

In [156]:
map_test = get_map(first_X_test)

100%|██████████| 16787/16787 [00:43<00:00, 386.56it/s]


In [168]:
map_test.head(20)

,player0_start,player1_start,map
0,5.0,6.0,4.0
1,12.0,13.0,6.0
2,13.0,12.0,6.0
3,4.0,15.0,3.0
4,14.0,5.0,4.0
5,10.0,8.0,5.0
6,8.0,10.0,5.0
7,10.0,8.0,5.0
8,12.0,13.0,6.0
9,3.0,11.0,2.0
